In [ ]:
import os
import numpy as np
import pandas as pd
#-------------------
import poseigen_seaside.basics as se
import poseigen_seaside.metrics as mex
import poseigen_chisel as chis

In [ ]:
datafolder = se.NewFolder('data_1')
os.chdir(os.getcwd() + datafolder )
#------------------------
tomain = './../'
hg38folder = tomain + 'hg38/'
#-------------------------
# we have a dataset_identifier file already here. 
data_source = pd.read_excel('Dataset_Identifiers.xlsx')
#-------------------------
bamfolder = se.NewFolder('bam')
peaksfolder = se.NewFolder('peaks')
#-------------------------
curfolder = se.NewFolder('currents')
procfolder = se.NewFolder(curfolder + 'processed')
fullfolder = se.NewFolder(curfolder + 'full') 
lowerfolder = se.NewFolder(curfolder + 'lowres') 
autosfolder = se.NewFolder(curfolder + 'autos')
winfolder = se.NewFolder(curfolder + 'win')

In [ ]:
# #Downloading all files
# path_A = 'https://www.encodeproject.org/files/'
# path_B = '/@@download/'
# path_C = '.bam'

# for index, row in data_source.iterrows():
#     newp = bamfolder + row['newid'] + path_C
#     if os.path.exists(newp) is False: 
#         se.download_url(path_A + row['id'] + path_B + row['id'] + path_C, newp)

In [ ]:
lowestres = 10
CTreso = 50
window = 200 #average size of fragments abouts

# YY1 Peaks

In [ ]:
# !macs3 callpeak -t `ls ./bam/YY1*.bam` -c `ls ./bam/Input_BGG*.bam` -f BAM -g hs -q 0.001 -n YY1 --outdir ./peaks/YY1 --call-summits

# YY1 and Histone Tracks

In [10]:
# I want pairs of its target and its input. 

subjects = ['yy1', 'histone']

pairs = np.concatenate([data_source[data_source['target_type'] == tar][['target', 'control']].drop_duplicates().to_numpy()
         for tar in subjects])

targets = np.unique(pairs)

print(pairs)
print(targets)

[['YY1' 'Input_BGG']
 ['H3K4me3' 'Input_AKY']
 ['H3K27ac' 'Input_AKY']
 ['H3K27me3' 'Input_AKY']
 ['H3K4me1' 'Input_AKY']
 ['H3K36me3' 'Input_AKY']
 ['H3K9me3' 'Input_AKY']
 ['H3K9ac' 'Input_AKY']
 ['H3K4me2' 'Input_AKY']
 ['H4K20me1' 'Input_AKY']
 ['H2AFZ' 'Input_AKY']
 ['H3K79me2' 'Input_AKY']]
['H2AFZ' 'H3K27ac' 'H3K27me3' 'H3K36me3' 'H3K4me1' 'H3K4me2' 'H3K4me3'
 'H3K79me2' 'H3K9ac' 'H3K9me3' 'H4K20me1' 'Input_AKY' 'Input_BGG' 'YY1']


In [10]:
# predictd stuff

# !macs3 predictd -i `ls ./../../data/!ENCODE/bam/YY1_Rep1.bam` -g hs --outdir ./peaks/YY1/predictd/Rep1
# !macs3 predictd -i `ls ./../../data/!ENCODE/bam/YY1_Rep2.bam` -g hs --outdir ./peaks/YY1/predictd/Rep2

# !macs3 predictd -i `ls ./../../data/!ENCODE/bam/Input_BGG_Rep1.bam` -g hs --outdir ./peaks/Input_BGG/predictd/Rep1
# !macs3 predictd -i `ls ./../../data/!ENCODE/bam/Input_BGG_Rep2.bam` -g hs --outdir ./peaks/Input_BGG/predictd/Rep2
# !macs3 predictd -i `ls ./../../data/!ENCODE/bam/Input_BGG_Rep3.bam` -g hs --outdir ./peaks/Input_BGG/predictd/Rep3

# !for r in Rep1 Rep2; do for h in H3K4me3 H3K27ac H3K27me3 H3K4me1 H3K36me3 H3K9me3 H3K9ac H3K4me2 H4K20me1 H2AFZ H3K79me2; do macs3 predictd -i `ls ./../../data/!ENCODE/bam/"$h"_"$r"*.bam` -g hs --outdir ./peaks/$h/$r/predictd; done; done
# !for r in Rep1 Rep2 Rep3; do for h in Input_AKY Input_BGG; do macs3 predictd -i `ls ./../../data/!ENCODE/bam/"$h"_"$r"*.bam` -g hs --outdir ./peaks/$h/$r/predictd; done; done

In [30]:
# for index, row in data_source[data_source['target'].isin(targets)].iterrows(): 
#     target, rep, newid, read, predd = row['target'], row['rep'], row['newid'], row['read'], row['predictd']

#     paio, readl = (True, None) if read == 'Paired' else (False, int(predd))
#     print(readl)

#     fpath = fullfolder + target + '_' + str(rep) + '_' + 'full'
#     lrpath = lowerfolder + target + '_' + str(rep) + '_' + 'lowres'
#     autopath = autosfolder + target + '_' + str(rep) + '_' + 'autos'

#     if os.path.exists(fpath + '.p') is False: 

#         full = chis.Bam2Current(bamfolder + newid + '.bam', hg38_sizes, hg38_id, select_BS_ids = hg38_basic_id, 
#                         newreso = lowestres, resomode = np.mean, stranded = False, 
#                         read_length = readl, paired = paio, 
#                         make_index = True, dtype = np.float32)
    
#         se.PickleDump(full, fpath)
    
#     if os.path.exists(lrpath + '.p') is False: 

#         full = se.PickleLoad(fpath)

#         lower = chis.LowerResCurrent(full, lowestres, CTreso, resomode = np.mean, dtype = np.float32)

#         se.PickleDump(lower, lrpath)
    
#     if os.path.exists(autopath + '.p') is False: 

#         lower = se.PickleLoad(lrpath)

#         autoz = chis.CurrentSelect(lower, hg38_basic_id, hg38_auto_ids)

#         se.PickleDump(autoz, autopath)

#     print(f'finished {target}, {rep}')

In [31]:
# # for each file we smooth by mean, then we remove blacklist regions, then we scale to total score. 

# hg38_basic_BLC_LR50 = se.PickleLoad(hg38folder + 'hg38_basic_BLC_LR50') #0s are bad

# for index, row in data_source[data_source['target'].isin(targets)].iterrows(): 
#     target, rep, newid, read, predd = row['target'], row['rep'], row['newid'], row['read'], row['predictd']

#     autopath = autosfolder + target + '_' + str(rep) + '_' + 'autos'

#     winpath = winfolder + target + '_' + str(rep) + '_' + 'win'

#     if os.path.exists(winpath + '.p') is False:

#         autos = se.PickleLoad(autopath)
        
#         win = chis.CurrentWindower(autos, reso = CTreso, mode = np.mean, window = window, 
#                                 center = 0, extend = 0, dtype = np.float32, indiv = True)
        
#         win2 = chis.CurrentModifier(win, hg38_basic_BLC_LR50,
#                                 mode = np.multiply, dtype = np.float32, single = False)
        
#         se.PickleDump(win2, winpath)

In [35]:
# for t in targets: 
#     reps = [1,2,3] if 'Input' in t else [1,2]

#     print(t)

    # targ_vals = chis.Currents2Current([se.PickleLoad(winfolder + t + '_' + str(rep) + '_' + 'win')
    #                                 for rep in reps]) ####################
    
    # targ_vals_scaled = chis.CurrentScaler(targ_vals, rounds = 1, dtype = np.float64)

    # targ_vals_mean = chis.CurrentMerger(targ_vals, mode = np.mean, dtype = np.float32)

    # for d, s in zip([targ_vals_scaled, targ_vals_mean], 
    #                 [t + '_scaled', t + '_mean']):
        
    #     se.PickleDump(d, procfolder + s)

## Over Inputs

In [ ]:
for papa in pairs: 

    targ_mean, inp_mean = [se.PickleLoad(procfolder + didi + '_mean') for didi in papa]

    targ_overinp_mode = [mex.BetaPrime_Mode(x, y) for x,y in zip(targ_mean, inp_mean)]

    targ_overinp_exp = [np.hstack([x,y]) for x,y in zip(targ_mean, inp_mean)]

    [se.PickleDump(x, procfolder + papa[0] + '_' + y) for x,y in zip([targ_overinp_mode, targ_overinp_exp], 
                                                                     ['mode', 'exp'])]

In [ ]:
inp_mean_vals = np.vstack(inp_mean).reshape(-1)

inp_mean_vals_filt = inp_mean_vals[(inp_mean_vals > 0) * (inp_mean_vals < 10)]

#plt.hist(inp_mean_vals_filt, bins = 50);

In [ ]:
histones = data_source[data_source['target_type'] == 'histone']['target'].unique()

array(['H3K4me3', 'H3K27ac', 'H3K27me3', 'H3K4me1', 'H3K36me3', 'H3K9me3',
       'H3K9ac', 'H3K4me2', 'H4K20me1', 'H2AFZ', 'H3K79me2'], dtype=object)

In [ ]:
his_mode = chis.Currents2Current([se.PickleLoad(procfolder + h + '_mode') for h in histones])

se.PickleDump(his_mode, procfolder + 'his_mode')

In [ ]:
his_inp_means = chis.Currents2Current([se.PickleLoad(procfolder + h + '_mode') for h in histones])

se.PickleDump(his_mode, procfolder + 'his_mode')